In [273]:
from jupyterthemes.stylefx import set_nb_theme
set_nb_theme('gruvboxd')

In [274]:
%run Libraries.ipynb

In [423]:
stocks = pd.read_csv('stocks.csv')
customers = pd.read_csv('customers.csv')
orders_1 = pd.read_csv('Orders(1).csv', encoding='latin-1', on_bad_lines='skip') 
order_items = pd.read_csv('order_items.csv')
products = pd.read_csv('products.csv')

In [424]:
products = products.index.to_frame().reset_index()['index'].to_frame()['index'].str.split(';', expand=True).rename(columns={1: 'product_name',
                                                                                                                           0: 'product_id'})

In [425]:
order_item = order_items.index.to_frame()[0].reset_index()['level_0'].to_frame()['level_0'].str.split(';', expand=True).drop(5, axis=1)

In [426]:
order_i = order_items['order_id;item_id;product_id;quantity;list_price;discount'].reset_index()['order_id;item_id;product_id;quantity;list_price;discount']

In [427]:
order_items = pd.concat([order_item, order_i], axis=1)

In [428]:
order_items.rename(columns={2: 'product_id',
                            3: 'quantity',
                            4: 'list_price',
                            'order_id;item_id;product_id;quantity;list_price;discount': 'discount'},
                            inplace=True)

In [429]:
stocks = stocks['store_id;product_id;quantity'].str.split(';', expand=True).rename(columns={0: 'store_id',
                                                                                  1: 'product_id',
                                                                                  2: 'quantity'}) 

In [430]:
customers = customers['customer_id;first_name;last_name;phone;email;street;city;state;zip_code'].str.split(';', expand=True)
customers = customers.rename(columns={0: 'customer_id',
                         1: 'customer_name'})

In [431]:
orders_1 = orders_1['orderid;custid;empid;orderdate;requireddate;shippeddate;shipperid;freight;shipname;shipcity;shipregion;shippostalcode;shipcountry;shipaddress'].str.split(';', expand=True)

In [432]:
orders_1 = orders_1.index.to_frame()[0].str.split(';', expand=True).reset_index(drop=True).iloc[:,:8]
orders_1 = orders_1.rename(columns={0: 'order_id',
                        1: 'customer_id',
                        7: 'freight'})

### Task 1:

In [433]:
pysqldf('''Select order_id, 
                customer_name,
            dense_rank() over (partition by customer_id order by cast(freight as float) desc) freight_ranking,
            freight
            from
            (Select * from orders_1
            left join customers on orders_1.customer_id=customers.customer_id
            )'''
       )

,order_id,customer_name,freight_ranking,freight
0,10835,Debra,1,69
1,10692,Debra,2,61
2,10952,Debra,3,40
3,10643,Debra,4,29
4,10702,Debra,5,23
...,...,...,...,...
649,10792,Marvin,3,23
650,10998,Marvin,4,20
651,10870,Marvin,5,12
652,11044,Marvin,6,8


### Task 2:

In [434]:
pysqldf('''Select product_id, CASE WHEN cast(quantity as float)>avg(cast(quantity as int)) then 'Stock Normal' 
            when cast(quantity as float)=avg(cast(quantity as int)) then 'Stock Go out'
            when cast(quantity as float)<avg(cast(quantity as int)) then 'Stock Out'
            end stock_condition
            from stocks
            group by product_id
            limit 15''')

,product_id,stock_condition
0,1,Stock Normal
1,10,Stock Out
2,100,Stock Out
3,101,Stock Normal
4,102,Stock Out
5,103,Stock Normal
6,104,Stock Normal
7,105,Stock Normal
8,106,Stock Normal
9,107,Stock Go out


### Task 3:

In [435]:
#Example
pysqldf('''Select product_id,
            list_price,
            discount
            from order_items
            where product_id=35''')

,product_id,list_price,discount
0,35,832,2.0
1,35,832,2.0
2,35,832,1.0
3,35,832,1.0
4,35,832,5.0
5,35,832,7.0
6,35,832,1.0
7,35,832,2.0
8,35,832,1.0
9,35,832,5.0


In [491]:
#Let's find out how many products with price and discount are there
pysqldf('''Select max(rows) number_of_products from

            (Select row_number() over (order by product_name) as rows,
            product_name
            from order_items 
            left join products on order_items.product_id=products.product_id
            group by product_name)''')

,number_of_products
0,278


In [496]:
pysqldf('''Select product_name,
            cast(avg(list_price) as float) as average_price,
            round(cast(avg(case when discount<>'None' then discount else 0 end) as float),1) as average_discount,
            count(1) number_of_purchases
            from order_items
            left join products on order_items.product_id=products.product_id
            group by product_name
            order by number_of_purchases desc
            ''')

,product_name,average_price,average_discount,number_of_purchases
0,Electra Townie Original 21D - 2016,549.0,3.7,193
1,Electra Cruiser 1 (24-Inch) - 2016,269.0,3.9,193
2,Electra Townie Original 7D EQ - 2016,599.0,3.8,185
3,Electra Girl's Hawaii 1 (16-inch) - 2015/2016,269.0,3.5,180
4,Surly Ice Cream Truck Frameset - 2016,469.0,3.8,110
...,...,...,...,...
273,Electra Cruiser 1 Tall - 2016/2018,269.0,1.0,1
274,Electra Amsterdam Royal 8i - 2017/2018,1259.0,1.0,1
275,"""Electra Treasure 3i 20"""" - 2018""",369.0,2.0,1
276,"""Electra Superbolt 3i 20"""" - 2018""",369.0,5.0,1


In [512]:
pysqldf('''Select product_name,
            round(avg(cast((list_price-discount) as float)),1) as avg_price_minus_discount,
            count(1) as number_of_purchases
            from order_items
            left join products on order_items.product_id=products.product_id 
            group by product_name
            ''')

,product_name,avg_price_minus_discount,number_of_purchases
0,"""Electra Girl's Hawaii 1 16"""" - 2017""",294.9,49
1,"""Electra Starship 1 16"""" - 2018""",275.7,3
2,"""Electra Superbolt 1 20"""" - 2018""",314.0,1
3,"""Electra Superbolt 3i 20"""" - 2018""",364.0,1
4,"""Electra Treasure 1 20"""" - 2018""",314.7,6
...,...,...,...
273,Trek X-Caliber 8 - 2017,995.2,24
274,Trek X-Caliber 8 - 2018,997.5,2
275,Trek X-Caliber Frameset - 2018,1494.3,3
276,Trek XM700+ - 2018,3497.6,5
